# Discrete Log Problem

For a group $G$ , given an element g∈G﻿  and $g^k=b$ , it is generally a difficult task \(approximately brute\-force over all elements\) to determine the exponent  $k$.

The larger the order of $g$, the more difficult the problem becomes.

Clearly, $k$ is not greater than the order of the group.


# Diffie Hellman



In [3]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.hkdf import HKDF

#### Generate some parameters. These can be reused.



In [4]:
parameters = dh.generate_parameters(generator=2, key_size=2048)

#### Generate a private key for use in the exchange.



In [5]:
bob_private_key = parameters.generate_private_key()

#### In a real handshake the node is a remote client. For this example we'll generate here another local private key though. Note that in a DH handshake both clients must agree on a common set of parameters.



In [6]:
alice_private_key = parameters.generate_private_key()

In [7]:
bob_shared_key = bob_private_key.exchange(alice_private_key.public_key())

#### Perform key derivation.



In [8]:
derived_key = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data',
).derive(bob_shared_key)

#### And now we can demonstrate that the handshake performed in the opposite direction gives the same final value



In [9]:
alice_shared_key = alice_private_key.exchange(
    bob_private_key.public_key()
)
same_derived_key = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data',
).derive(alice_shared_key)

print(derived_key.hex())
print(derived_key == same_derived_key)

e1cb73aaf0c4d0d7938183afa70024cfdaf3685b5d8464b5001dde333f80e96d
True


##### **Exercise**: AES Message Encryption

Alice and Bob  securely exchange a shared key using the Diffie\-Hellman protocol. The next step woule be to use this shared key to encrypt a secret message from Alice to Bob using AES in GCM mode, which provides both encryption and authentication. 

**Hint**: You'll need to generate a random 12\-byte nonce \(also called IV\) for the encryption process, and include the authentication tag during encryption. Then, simulate Bob decrypting the message using the same key, nonce, and tag, and print out the original, encrypted, and decrypted versions to confirm it worked.



In [ ]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms
from Crypto.Cipher import Salsa20

aesgcm = AESGCM(same_derived_key)
salsa_key = b'\x01' * 32
salsa_nonce = b'\x02' * 16

nonce = Salsa20.new(key=salsa_key, nonce=salsa_nonce).encrypt(b"\x00" * 12)
plaintext = b"Hello Bob, this is a top-secret message from Alice."
ciphertext_and_tag = aesgcm.encrypt(nonce, plaintext, None)


ciphertext = ciphertext_and_tag[:-16]
tag = ciphertext_and_tag[-16:]

print("Original plaintext:", plaintext)
print("Nonce (hex):", nonce.hex())
print("Ciphertext (hex):", ciphertext.hex())
print("Tag (hex):", tag.hex())


aesgcm_bob = AESGCM(aes_key)

decrypted = aesgcm_bob.decrypt(, ciphertext_and_tag, None)
print("Decrypted plaintext:", decrypted)


NameError: name 'nonce' is not defined

##### **Exercise**: ElGamal Encryption

- Alice pk: $(x)$, pubk: $(G,g,g^x)$
- Bob enctypts: $(c_1=g^y, c_2=m(g^x)^y)$
- Alice decrypts: $m = c_2c_1^{-x}$



###

